In [1]:
import os

import torch
import torch.nn as nn
import torchvision
from tqdm import tqdm

import pickle
from matplotlib import pyplot as plt

## 1. Implement LeNet-5

- 在Week3的实验中，你已经学会了如何使用PyTorch内部封装好的模块，实现一个多层MLP模型。

- 在本周的实验任务Task1中，我们将：
  1. 继续利用PyTorch的内置神经网络模块（torch.nn.Module的子类）实现一个卷积神经网络LeNet5；
  2. 在实现好的LeNet5模型上，利用PyTorch的内置优化器实现模型的训练；
  3. 评估训好的模型的预测性能，并将其保存、以便本周任务Task2的调用。

- 具体实验步骤如下：
  1. 将代码文件（Python文件与Notebook文件）上传到服务器端根目录；
  2. 依照提示，完成Python文件中的TODO内容：
     - 一个LeNet5模型结构的定义
     - 该模型的前向传播的实现
  3. 执行代码完成模型训练、测试和保存。
     - **正确实现，测试的accuracy应该高于93%**

### 超参数定义、数据集准备

In [2]:
from Week567_General_Code_Question import LeNet5, load_mnist

batch_size = 128
lr = 0.01
epoch = 10
train_loader, test_loader = load_mnist(batch_size=batch_size)

100%|██████████| 9912422/9912422 [00:02<00:00, 4366577.35it/s]


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 52621934.76it/s]

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 4013030.02it/s]


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 647404.63it/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



### 模型的准备

请你在文件Week567_General_Code_Question.py的LeNet5类中实现两个TODO内容：
- 一个LeNet5模型结构的定义
- 该模型的前向传播的实现

下面是一些供你参考/可能用到的API函数：

- torch.nn.Conv2d(*in_channels*, *out_channels*, *kernel_size*, *stride=1*, *padding=0*, *bias=True*) [link](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html)
  - in_channels: 输入网络层的通道数
  - out_channels: 输出网络层的通道数
  - kernel_size: 卷积核的边长
  - stride: 卷积操作（滑动窗口）的步长
  - padding: 输入两端补零的数目
- torch.nn.MaxPool2d(*kernel_size*) [link](https://pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html)
  - kernel_size: 池化操作的窗口边长
- torch.nn.Linear(*in_features*, *out_features*, *bias=True*) [
  Link](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html)
  - in_features: 输入网络层的特征维度
  - out_features: 输出网络层的特征维度
- torch.nn.ReLU() [Link](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html)
  - 常用的激活函数
- torch.Tensor.reshape(*shape*) [Link](https://pytorch.org/docs/stable/generated/torch.Tensor.reshape.html)
  - shape: 当前tensor希望修改为的形状，如(2, 2)或(-1, 3)
    - -1指该维度大小根据原数据维度大小和其它给定维度大小计算得到，至多可以给一个-1

In [3]:
model = LeNet5()

### 定义损失函数、优化算法

我们仍然使用上周的CrossEntropyLoss，以及SGD优化器。

In [4]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

### 模型的训练

In [5]:
model.train()

for e in range(epoch):
    t = tqdm(train_loader)
    for img, label in t:
        optimizer.zero_grad()
        pred = model(img)
        loss = criterion(pred, label)
        
        loss.backward()
        optimizer.step()

        t.set_postfix(epoch=e, train_loss=loss.item())

100%|██████████| 469/469 [00:11<00:00, 41.51it/s, epoch=9, train_loss=0.103] 


### 模型的测试

- torch.argmax(*input*, *dim*, *keepdim=False*) [Link](https://pytorch.org/docs/stable/generated/torch.argmax.html)
  - input: 计算基于的tensor
  - dim: 希望按哪个维度求max下标

**如果LeNet5实现正确，测试的accuracy应该高于93%**

In [6]:
model.eval()

correct_cnt, sample_cnt = 0, 0

t = tqdm(test_loader)
for img, label in t:
    pred = model(img)
    pred_label = pred.argmax(dim=1)
    
    correct_cnt += (pred_label == label).sum().item()
    sample_cnt += pred_label.shape[0]

    t.set_postfix(test_acc=correct_cnt/sample_cnt)

100%|██████████| 79/79 [00:01<00:00, 64.36it/s, test_acc=0.968]


### 模型的保存

我们将完成训练的模型保存到服务器的model/目录下，方便Task2的使用。

ModelScope服务器端无法长久保存文件，因此**请及时下载、本地保存你完成的代码，以及模型的参数文件（model/plt.pt）**。

In [7]:
if not os.path.exists('model/'):
    os.mkdir('model/')

torch.save(model.state_dict(), 'model/lenet5.pt')